<a href="https://colab.research.google.com/github/hereagain-Y/TCR_VAE/blob/main/Simulation_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import itertools


In [4]:
def getSampleIndex(sample):
    sample_list = list( set(sample) )
    sample_array = np.array(sample)
    sample_index = []
    for s in sample_list:
        sample_index.append( np.where(sample_array==s)[0] )
    return sample_list, sample_index


In [5]:
class AttentionNetwork(nn.Module):
    def __init__(self, n_input_features=16):
        super(AttentionNetwork, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        self.Tanh = nn.Tanh()
        
    
    def forward(self, x):
        attention_weights = self.Tanh( self.linear(x) )
        return attention_weights
        
class OutputNetwork(nn.Module):
    def __init__(self, n_input_features=16, n_output_features=1):       
        super(OutputNetwork, self).__init__()                
        self.linear = nn.Linear(n_input_features, n_output_features)
    
    def forward(self, inputs):        
        predictions = torch.sigmoid( self.linear(inputs) )
        return predictions
        
class myNet(nn.Module):
    def __init__( self, n_input_features=16, attention_network = AttentionNetwork(n_input_features=16), output_network = OutputNetwork(n_input_features=16) ):       
        super(myNet, self).__init__()
        self.n_input_features = n_input_features
        self.attention_nn = attention_network       
        self.output_nn = output_network
        
       
    def forward(self, sequence, sample, count):
        """ 
        
        Parameters
        n_sequences_per_bag: torch.Tensor
            Number of sequences per bag as tensor of dtype torch.long and shape (n_samples,)
        
        Returns
        ----------
        predictions: torch.Tensor
            Prediction for bags of shape (n_samples, n_outputs)
        """     
        #sequence should be np.array, so is sample
        # Calculate attention weights f() before softmax function for all bags in mb (shape: (d_k, 1))
        attention_weights = self.attention_nn(sequence) # 1*N
        #we multiply original attention weight by count
        count_col_vec = count.reshape(-1,1) # N*1 
        attention_weights = attention_weights * count_col_vec
        sample_list, sample_index = getSampleIndex(sample)
        sample_vec = []
        attention_weights_after_softmax = torch.clone( attention_weights )
        #print(attention_weights)
        for i in range(len(sample_list)):
            sample_index_this = sample_index[i]
            seq_this = sequence[ sample_index_this ] # get all seqs for that person  n*16
            #turn count from row vec to a column vec
            attention_weights_this = attention_weights[ sample_index_this ] # get attentions for that person  ?? is a row or colunmn vector?
            attention_weights_this = torch.softmax(attention_weights_this, dim=0)
            #each element of sequence is a row vector, attention weight is a column vector (if it is the result from attention network)
            sample_vec_this = seq_this * attention_weights_this #N*16 
            #so we sum along column
            sample_vec.append(sample_vec_this.sum(dim=0))
            attention_weights_after_softmax[ sample_index_this ] = attention_weights_this
        
        sample_vec = torch.stack(sample_vec, dim=0)       
        # Calculate predictions (shape (N, n_outputs))
        predictions = self.output_nn(sample_vec)  
        Y_hat = torch.ge(predictions, 0.5).float() # return 0,1 
             
        return predictions, Y_hat,sample_list, attention_weights_after_softmax
    
def loss_function( label, prediction ):
    reproduction_loss = nn.functional.binary_cross_entropy(label, prediction, reduction='sum')
    return reproduction_loss

    
def calculate_classification_error(Y_hat,Y):
    Y = Y.float()
    error = 1. - Y_hat.eq(Y).cpu().float().mean().data

    return error, Y_hat  


In [6]:
cuda = False
DEVICE = torch.device("cuda" if cuda else "cpu")
n_feature = 16   
model = myNet(n_input_features=n_feature, attention_network = AttentionNetwork(n_input_features=n_feature), output_network = OutputNetwork(n_input_features=n_feature)).to(DEVICE)

In [7]:
import random

In [8]:

def gen_multi_list(amount, length):
    seqs = np.random.default_rng()
    return [seqs.random(length) for _ in range(amount)]

sequences = gen_multi_list(100,16) #10*10
counts =[]
for i in range(0,100):
    n=random.randint(1, 5)
    counts.append(n)


In [9]:
sequences = torch.from_numpy( np.array(sequences)).float()

In [10]:
counts = torch.from_numpy( np.array( counts ) ).float()
# person _index  10 people 
A = ['a','b','c','d','e','f','g','h','i','j']


samples=list(itertools.chain.from_iterable(itertools.repeat(x, 10) for x in A))



label =[]
for i in range(0,10):
    n=random.randint(0, 1)
    label.append(n)
labels = np.repeat(label,10)
  

sample_label_map = { i:j for (i,j) in zip(samples,labels) }


In [11]:
sample_label_map

{'a': 1,
 'b': 1,
 'c': 1,
 'd': 1,
 'e': 1,
 'f': 1,
 'g': 1,
 'h': 1,
 'i': 0,
 'j': 0}

In [12]:
from torch.optim import Adam
optimizer = Adam(model.parameters(), lr=1e-3)

In [13]:
from sklearn.metrics import accuracy_score

In [15]:
epochs = 10000
for ite in range(epochs):
    overall_loss =0
    train_error = 0    
    predictions,Y_hat,sample_list, attention_weights = model(sequences, samples, counts)
    #get true sample label, for now, use binary
    label_true = [ sample_label_map[s] for s in sample_list ]
    #column vector
    label_true = np.array( label_true ).reshape( (len(label_true),1) )
    label_true = torch.from_numpy( label_true ).float()
    #Y_hat.shape 10*1
    #print(label_true[:5])
    #print(Y_hat[:5])
    loss = loss_function(predictions, label_true)
    overall_loss +=loss.item()
    
    error, predicted_label = calculate_classification_error(Y_hat, label_true)
    train_error += error
    
    loss.backward()
    optimizer.step()
    if (ite % 1000 == 0):
      print('Train Set, Epoch: {}, Loss: {:.4f},Error: {:.4f}, Accuracy: {:.2f}%'.format(ite+1, overall_loss,train_error,accuracy_score(label_true, Y_hat)*100))
    
    #print(ite,loss)




Train Set, Epoch: 1, Loss: 7.6178,Error: 0.8000, Accuracy: 20.00%
Train Set, Epoch: 1001, Loss: 2.9629,Error: 0.2000, Accuracy: 80.00%
Train Set, Epoch: 2001, Loss: 0.8552,Error: 0.0000, Accuracy: 100.00%
Train Set, Epoch: 3001, Loss: 0.2604,Error: 0.0000, Accuracy: 100.00%
Train Set, Epoch: 4001, Loss: 0.0719,Error: 0.0000, Accuracy: 100.00%
Train Set, Epoch: 5001, Loss: 0.0345,Error: 0.0000, Accuracy: 100.00%
Train Set, Epoch: 6001, Loss: 0.0159,Error: 0.0000, Accuracy: 100.00%
Train Set, Epoch: 7001, Loss: 0.0077,Error: 0.0000, Accuracy: 100.00%
Train Set, Epoch: 8001, Loss: 0.0052,Error: 0.0000, Accuracy: 100.00%
Train Set, Epoch: 9001, Loss: 0.0018,Error: 0.0000, Accuracy: 100.00%


In [17]:
from google.colab import drive

In [18]:
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:

torch.save(model, '/content/drive/My Drive/DL/Prediction_model/simulation_100seq_1w_echo_train.apx')

Test on the model